In [122]:
import json, subprocess, re, pyodbc, os
from sqlalchemy import func, create_engine
from bs4 import BeautifulSoup 
import numpy as np # linear algebra 
from config import mongoConn, connstr
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 
import datetime as dt

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
There is no [win32] chromedriver for browser 92.0.4515 in cache
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Tim\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [44]:
with open('C:\\bom\\appsettings.json', encoding='utf-8-sig') as f:
    config = json.load(f)
pw = ''
for s in config['contexts']:
    if 'jira' in s['name']:
        pw=re.search('password,(.*);c:submit;', s['conn']).group(1)   

In [53]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(f"https://dayman.cyber-balance.com/jira/login.jsp")
browser.find_by_css('#login-form-username').fill('tim.kopp')
browser.find_by_css('#login-form-password').fill(pw)
browser.find_by_css('#login-form-submit').click()
html=browser.html

In [88]:
soup = BeautifulSoup(html, 'html.parser')
els = soup.select('.issue-table')   

In [101]:
df = pd.read_html( html )[0] 
df['task'] = df.Key + ' ' + df['Summary']
df = df.loc[ (df['Fix Version/s'] == '7.36') & (df['Summary'].str.contains('SAOP'))   ]
df.head()

,T,Key,Summary,P,Fix Version/s,Unnamed: 5,task
9,NaN,CS-8412,FY21 Annual SAOP Data Call,NaN,7.36,Actions,CS-8412 FY21 Annual SAOP Data Call
10,NaN,CS-8413,FY21 Annual SAOP Data Call: Section 1 - Genera...,NaN,7.36,Actions,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
11,NaN,CS-8414,FY21 Annual SAOP Data Call: Section 2 - Inform...,NaN,7.36,Actions,CS-8414 FY21 Annual SAOP Data Call: Section 2 ...
12,NaN,CS-8415,FY21 Annual SAOP Data Call: Section 3 - Inform...,NaN,7.36,Actions,CS-8415 FY21 Annual SAOP Data Call: Section 3 ...
13,NaN,CS-8416,FY21 Annual SAOP Data Call: Section 4 - System...,NaN,7.36,Actions,CS-8416 FY21 Annual SAOP Data Call: Section 4 ...


In [135]:
today = dt.datetime.today().strftime("%m-%d-%Y") 
sheet = str(today)
for d in pd.date_range(start=today, periods=5, freq='D' ):
    print( d)

2021-08-16 00:00:00
2021-08-17 00:00:00
2021-08-18 00:00:00
2021-08-19 00:00:00
2021-08-20 00:00:00


In [143]:
lst = [] 
tskcnt = 0
for d in pd.date_range(start=today, periods=5, freq='D'):
    tskcnt += 1
    for t in range(900, 1700, 100):  
        lst.append({'date': re.search('(.*)\s.*', str(d)).group(1), 'time': str(t) , 'task': df.iloc[tskcnt].task })
dff = pd.DataFrame(lst)
dff

,date,time,task
0,2021-08-16,900,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
1,2021-08-16,1000,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
2,2021-08-16,1100,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
3,2021-08-16,1200,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
4,2021-08-16,1300,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
5,2021-08-16,1400,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
6,2021-08-16,1500,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
7,2021-08-16,1600,CS-8413 FY21 Annual SAOP Data Call: Section 1 ...
8,2021-08-17,900,CS-8414 FY21 Annual SAOP Data Call: Section 2 ...
9,2021-08-17,1000,CS-8414 FY21 Annual SAOP Data Call: Section 2 ...


In [150]:
with pd.ExcelWriter('C:\\Users\\Tim\\Documents\\LOGS\\tasklog\\'+ sheet +'.xlsx') as writer:   
    dff.to_excel(writer, sheet_name=f'{sheet}')  